# Homework 4: Practical analysis with BioPython

For the homework, you are going to extend the code from the analysis of our FASTQ file in lectures 8 and 9.
Recall that the FASTQ file contains reads from a real sequencing run of influenza virus HA and NA genes.

The _actual sequences_ are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N barcode]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N barcode]-3'
    
The end of NA is:

    ...CACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ...CCGGATTTGCATATAATGATGCACCAT
    
The _sequencing reads_ (located in the FASTQ file) are from the reverse end of these actual sequences, so the first thing in the sequencing reads is the reverse complement of the barcode followed by the reverse complement of the constant sequence, etc.
The reads can originate from **either** HA or NA, and that will be distinguished by the most 3' end of the read.
But in our example exercise in class, we did not distinguish among reads matching to HA and NA, as we didn't even look far enough into the read to tell the identity.

For the homework, your goal is to write code that extends the material from lectures 8 and 9 to also distinguish between HA and NA.
This homework can be completed almost entirely by re-using code from lecture 9. You will need to set up your analysis to do the following:
 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern for HA and NA, and if so which one.
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.
 4. Determine the number and distribution of barcodes for HA and NA separately.

Please include code to address each of the following questions. Please include code comments to explain what your code is attempting to accomplish. Don't forget to include references to the sources you used to obtain your answer, including your classmates (if you are working in groups).  

In [1]:
#Import Packages
import re
import Bio.SeqIO

In [5]:
#Read in sequencing data(from Lecture 9)
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq'))

In [6]:
#Convert SeqRecords objects to a list of strings(Lecture 9)
seqreads_str = []
for seqrecord in seqreads:
    sequence = str(seqrecord.seq) # convert sequence part to string
    seqreads_str.append(sequence) # add string sequence to list

In [7]:
#Make a function to get R.C from each strong(Lecture 9)
def reverse_complement(seq, unk_partner = 'N'):
    """Get reverse complement of a DNA sequence.
    
    Aurguments
    -----------
    seq : str
        Uppercase DNA sequence.
        
    Returns
    -------
    str
        Reverse complement of the sequence in upper case.
        
    Example
    --------
    >>> reverse_complement('ATGCAC')
    'GTGCAT'
    
    """
    base_partner = {'A':'T', 'T':'A', 'C':'G', 'G':'C'} #Make Dictionary of basepairs
    rseq = '' #Make Empty String
    for a in seq: #loops over bases in sequence
        if a in base_partner:
            # look up the complementary base in the dictionary
            pair = base_partner[a]
            rseq = pair + rseq #build reverse comp string
        else:
            rseq = unk_partner + rseq
    return rseq 

In [ ]:
#Make Function to Determine if sequence is from HA or NA
def read_type(seqread, NAend='CACGATAGATAAATAATAGTGCACCAT', HAend='CCGGATTTGCATATAATGATGCACCAT',endlen=27):
    # get the reverse complement of the read using our previously made function
    reverse = reverse_complement(seqread) 
    
    # compile the barcode search pattern
    type_pattern = re.compile("(?P<type>[ATCG]{"+str(endlen)+"})")

    
    
    # search for the barcode pattern
    match = type_pattern.search(reverse)
    
    if match:
        barcode = match.group('type')
        
    else:
        return None 


In [ ]:
#Make function to read barcodes(lecture 9)
def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : str
        Nucleotide sequence matching UPSTREAM-BARCODE read in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
    Example
    -------
    >>> read_barcode('TTTTTTTTTTTTTTTTGCGGCCGCCT', bclen=16)
    'AAAAAAAAAAAAAAAA'
        
    """
    # get the reverse complement of the read
    reverse = reverse_complement(seqread)
    
    # compile the barcode search pattern
    barcode_pattern = re.compile(upstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    # search for the barcode pattern
    match = barcode_pattern.search(reverse)
    
    if match:
        barcode = match.group('barcode')
        return barcode
    else:
        return None

1. How many reads map to HA, and how many reads map to NA?

2. What is the HA barcode with the most counts (and how many counts)? Also answer the same question for NA.

    _Hint: you will need to find the key associated with the maximum value in your dictionary. There are many ways to do this._